In [1]:
import numpy as np
import pandas as pd
import re
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
import lda
import random
random.seed(1)
import string
import nltk
from textblob import TextBlob
from textblob import Word
import pickle
from nltk.tokenize import word_tokenize

In [2]:
stats = pd.read_pickle('/Users/boyaliu/Documents/UCD/LIN 127/final/stats_df.pickle')
game = pd.read_pickle('/Users/boyaliu/Documents/UCD/LIN 127/final/gaming_df.pickle')


In [3]:
cook = pd.read_pickle('/Users/boyaliu/Documents/UCD/LIN 127/final/cooking_df.pickle')

In [4]:
#stats = stats
#game = game[1:15000]

In [5]:
stats['body_title'] = stats['Body'] + stats['Title']

In [6]:
cook['body_title'] = cook['Body'] + cook['Title']

In [7]:
cook.shape

(16060, 3)

In [8]:
stats.shape

(92335, 3)

# Text Preprocessing

In [9]:
verb_exp = ['VB', 'VBZ', 'VBP', 'VBD','VBN','VBG']
stop_words = pd.read_pickle('/Users/boyaliu/Documents/UCD/LIN 127/final/stop_words.pickle.txt')
def lem_stop(row):
    row = row.lower()
    #remove numbers
    raw = re.sub('[0-9]+?', ' ', row) 
    # remove punctuation
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    raw = regex.sub(' ', raw)
    #clean out the characters left out after the above step, like we’re, I’m, It’s, i.e.
    raw = re.sub('( s )|( re )|( m )|( i e )',' ',raw) 
    # lementize
    row_t = TextBlob(raw)
    raw = []
    for word, pos in row_t.tags:
        if pos in verb_exp:
            word = Word(word)
            word = word.lemmatize("v")
        else:
            word = Word(word)
            word = word.lemmatize()
        raw.append(word)
    clean = ' '.join(raw)       
    # remove stop words
    cleaned_text = " ".join([word for word in word_tokenize(clean) if word not in stop_words])      
    return(cleaned_text)

In [10]:
stat_clean = stats['body_title'].map(lambda i: lem_stop(i))

In [11]:
cook_clean = cook['body_title'].map(lambda i: lem_stop(i))

In [14]:
with open('stat_clean','wb') as f:
    pickle.dump(stat_clean,f)
with open('cook_clean','wb') as f:
    pickle.dump(cook_clean,f)

# Topic Modeling for Stats Data

In [15]:
stat_clean_sub = stat_clean.sample(n = 20000, random_state=0, axis=None)

In [16]:


cvectorizer = CountVectorizer(min_df=4, max_features=10000, stop_words='english')
cvz = cvectorizer.fit_transform(stat_clean_sub)

n_topics = 10
n_iter = 2000
stat_lda_model_sub = lda.LDA(n_topics=n_topics, n_iter=n_iter, random_state = 0)
stat_X_topics_sub = stat_lda_model_sub.fit_transform(cvz)

n_top_words = 8
topic_summaries = []

topic_word = stat_lda_model_sub.topic_word_  # get the topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: data set feature cluster class model refcode dataset
Topic 1: distribution probability parameter random function variable normal likelihood
Topic 2: model refcode variable regression linear effect predictor logistic
Topic 3: test data hypothesis difference sample refcode result significant
Topic 4: reflink network question learn algorithm problem paper neural
Topic 5: matrix correlation vector covariance component variable variance pca
Topic 6: time data series model plot refcode point fit
Topic 7: group data analysis variable effect factor measure subject
Topic 8: number probability time event question data problem user
Topic 9: sample mean standard interval error confidence estimate size


# Topic Modeling for Cooks Data

In [17]:
cvectorizer = CountVectorizer(min_df=4, max_features=10000, stop_words='english')
cvz = cvectorizer.fit_transform(cook_clean)

n_topics = 10
n_iter = 2000
cook_lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter, random_state = 0)
cook_X_topics = cook_lda_model.fit_transform(cvz)

n_top_words = 8
topic_summaries = []

topic_word = cook_lda_model.topic_word_  # get the topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: reflink food question make good buy knife difference
Topic 1: cook rice water potato make boil soup bean
Topic 2: cook meat chicken beef roast pork steak time
Topic 3: store eat day fridge long safe freeze food
Topic 4: pan oven oil heat cook temperature iron pot
Topic 5: make bake cake oven recipe pie pizza crust
Topic 6: sauce oil make taste add tomato pepper onion
Topic 7: make milk cream sugar butter chocolate cheese recipe
Topic 8: egg bread flour recipe dough make bake yeast
Topic 9: make coffee tea fruit juice water taste lemon
